In [1]:
import numpy as np
import pandas as pd

from vajeED_1_podatki import *

# Odkrivanje enačb, vaje 1

# 1. del: linearna regresija

Ena najpreprostejših in pogosto precej učinkovitih metod za odkrivanje enačb je redka linearna regresija. Omejena je na enačbe, linearne v parametrih, omogoča nam pa dodajanje členov s poljubnimi oblikami funkcij. Pri linearni regresiji iščemo model oblike $\hat{y}_i = \sum\limits_{k=0}^K \beta_k X_{ik}$, kjer je $\vec{\beta}$ vektor koeficientov, $X$ pa matrika podatkov. Če minimiziramo kvadratno napako $\sum_i (\hat{y}_i - y_i)^2$, se da izpeljati analitično rešitev za koeficiente:

$$\vec{\beta} = (X^TX)^{-1}X^T \vec{y}$$

Pri uporabi linearne regresije za odkrivanje enačb so ključne dodatne spremenljivke, ki jih zgeneriramo. To so lahko višji redi spremenljivk, produkti spremenljivk, logaritmi, trigonometrične funkcije, itd. 

Za razliko od navadnega strojnega učenja nas pri odkrivanju enačb ne zanimajo preveč napovedi, temveč sam model - najboljša enačba, ki smo jo uspeli odkriti. Ker želimo čim bolj razumljive enačbe, je dobro iz končne enačbe odstraniti člene z zelo majhnimi koeficienti. Berljivost enačbe lahko dodatno izboljšamo tako, da v stringu vrednosti koeficientov zaokrožimo na manjše število decimalnih mest.

1.1 Napiši funkcijo za linearno regresijo. Sprejme naj:
- X: tabela z N vrsticami, ki ustrezajo N učnim primerom, ter K stolpci, ki ustrezajo K spremenljivkam (v splošnem bodo to originalne spremenljivke plus neko število dodatnih členov), imena stolpcev pa povejo, za kateri člen gre (np. "log(x-y)")
- y: vektor z N vrednostmi, ki ustrezajo levi strani enačbe.
- meja: opcijski argument, ki pove, pod katero vrednostjo koeficienta člen odstranimo iz enačbe.

Funkcija naj vrne:
- najdeno enačbo v obliki stringa (npr. "0.32 x + 0.10 x**2 - 2.02 sin(y)")


In [38]:
def linearna_regresija(X, y, meja=1e-2):
    cleni = list(X)
    X = np.array(X)
    XX = np.dot(np.transpose(X), X)
    Xy = np.dot(np.transpose(X), y)
    beta = np.linalg.solve(XX, Xy)
    rtrstr = ""
    for i, el in enumerate(cleni):
        if np.abs(beta[i]) <= meja:
            continue
        
        rtrstr += f" {beta[i]:+.2f}{el} "
    return print(rtrstr)

In [35]:
X = np.array([[1, 2], [3, 5]])
y = np.array([1, 2])
X = pd.DataFrame(X, columns=['x1', 'x2'])
linearna_regresija(X, y)


-1.00x1+1.00x2


1.2 Preizkusi svojo funkcijo za linearno regresijo na primeru odkrivanja energijskega zakona $E = mgh + \frac{1}{2}mv^2$. Za generiranje podatkov lahko uporabiš funkcijo **generiraj_energijski_zakon** iz datoteke **vajeED_1_podatki.py**. Podaš željeno število primerov (recimo 100) ter željeno stopnjo multiplikativnega šuma.

Zgenerirati bo potrebno tudi dodatne člene. To lahko sprogramiraš ročno, lahko pa uporabiš kakšno od funkcij iz scikit-learn. Za generiranje polinomskih členov je uporabna **sklearn.preprocessing.PolynomialFeatures**.

Kako dobro deluje navadna linearna regresija? Preizkusi vsaj tri nivoje šuma - smiselne vrednosti so recimo 0, 0.001, 0.01 in 0.1.

In [43]:
podatki_energija = generiraj_energijski_zakon(100, sum=0.001)


y = podatki_energija["E"]
X = podatki_energija.drop(["E"], axis=1)

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)
X = poly.fit_transform(X)
novi_cleni = poly.get_feature_names_out()

podatki_energija_razsirjeni = pd.DataFrame(X, columns=novi_cleni)

linearna_regresija(podatki_energija_razsirjeni, podatki_energija["E"], meja=1e-1)

 +9.79m h  +0.49m v^2 


1.3 Delovanje pri zašumljenih podatkih lahko včasih izboljšamo z regularizacijo. 
Pri redki (ang. sparse) regresiji želimo obržati vrednosti koeficientov nizke. Visoke vrednosti kaznujemo tako, da v funkcijo napake dodamo regularizacijski člen (ang. ridge regression):  $\sum_i (\hat{y}_i - y_i)^2 + \lambda \sum_k \beta_k^2$, kjer parameter $\lambda$ določa nivo regularizacije. Optimizacijski problem se da še vedno rešiti analitično:

$$ \vec{\beta} = (X^TX + \lambda I)^{-1}X^T \vec{y} $$ 

Napiši funkcijo za ridge regresijo. Sprejme naj:
- X: tabela z N vrsticami, ki ustrezajo N učnim primerom, ter K stolpci, ki ustrezajo K spremenljivkam (v splošnem bodo to originalne spremenljivke plus neko število dodatnih členov), imena stolpcev pa povejo, za kateri člen gre (np. "log(x-y)")
- y: vektor z N vrednostmi, ki ustrezajo levi strani enačbe.
- lambda: regularizacijski parameter, dobro vrednost za dani problem je treba določiti empirično (s poskušanjem)
- meja: opcijski argument, ki pove, pod katero vrednostjo koeficienta člen odstranimo iz enačbe


Funkcija naj vrne:
- najdeno enačbo v obliki stringa (npr. "0.32 x + 0.10 x**2 - 2.02 sin(y)")

Funkcijo za ridge regresijo preveri na podatkih za energijski zakon.

In [44]:
def ridge_regresija(X, y, lam=1, meja=1e-2):
    cleni = list(X)
    X = np.array(X)
    XX = np.dot(np.transpose(X), X)
    XX = np.add(XX, lam * np.identity(XX.shape[0]))
    Xy = np.dot(np.transpose(X), y)
    beta = np.linalg.solve(XX, Xy)
    rtrstr = ""
    for i, el in enumerate(cleni):
        if np.abs(beta[i]) <= meja:
            continue
        rtrstr += f" {beta[i]:+.2f}{el} "
    return print(rtrstr)


In [55]:
ridge_regresija(podatki_energija_razsirjeni, podatki_energija["E"], lam=1, meja=3e-1)

 +1.10m  +1.04h  +0.53m^2  +2.47m h  +0.32m v  +0.54h^2  +1.98m^2 h  +1.92m h^2  +0.83m h v 


1.4 Ridge regresija za ta problem očitno ne deluje najboljše. V resnici je za odkrivnaje enačb bolj primerna regularizacija z normo L1:
$\sum_i (\hat{y}_i - y_i)^2 + \lambda \sum_k |\beta_k|$.
Taki linearni regresiji rečemo Lasso in žal nima analitične rešitve, zato moramo optimalne vrednosti parametrov $\beta_k$ iskati z numerično minimizacijo (recimo **scipy.optimize.minimize**).

Napiši funkijo za Lasso regresijo z istimi vhodi in izhodi kot v 1.3 ter jo preizkusi na podatkih za energijski zakon. Ali deluje bo


In [76]:
from scipy.optimize import minimize
from sklearn import linear_model

def lasso_regresija_skl(X, y, lam=1, meja=1e-2):
    clf = linear_model.Lasso(alpha=0.01)
    clf.fit(X, y)
    cleni = list(X)
    beta = clf.coef_
    rtrstr = ""
    for i, el in enumerate(cleni):
        if np.abs(beta[i]) <= meja:
            continue
        rtrstr += f" {beta[i]:+.2f}{el} "
    print(rtrstr)

def lasso_regresija(X, y, lam=1, meja=1e-2):
    imena = X.columns

    def f(beta):
        yhat = X.dot(beta)
        return np.sum((yhat-y)**2) + lam*np.sum(np.abs(beta))
    beta = minimize(f, np.random.random(X.shape[1]))["x"]
    
    izraz = ""
    for i,b in enumerate(beta):
        if b > meja:
            if len(izraz) > 0:
                izraz += " + "
            izraz +=  f"{b:.3f}*{imena[i]}"
    return izraz

In [78]:
lasso_regresija_skl(podatki_energija_razsirjeni, podatki_energija["E"], lam=1, meja=2e-1)
lasso_regresija(podatki_energija_razsirjeni, podatki_energija["E"], lam=1, meja=2e-1)

 +9.68m h  +0.30m v 


'9.752*m h + 0.222*m v'

DODATNO: Odkrivanje enačb z linearno regresijo lahko preizkusiš tudi na drugih podatkih v datoteki **vajeED_1_podatki.py**.

DODATNO-2: Na vajah smo spoznali osnovno implementacijo redke regresije za odkrivanje enačb. Za bolj resne implementacijo si lahko pogledaš knjižnico SINDy, ki je namenjena predvsem odkrivanju sistemov diferencialnih enačb, ki opisujejo dinamične sisteme: https://github.com/dynamicslab/pysindy.

In [ ]:
import pysindy as ps





## 2. del - BACON
Na predavanjih smo spoznali enega prvih algoritmov za odkrivanje enačb - BACON. Ta algoritem je koristen za odkrivanje enačb, ki vključujejo samo množenje in deljenje. 

![algoritem BACON](bacon.png)

Zapiši funkcijo, ki sprejme tabelo T s stolpci $x_i$, $1 \le i \le m$ in najde zvezo $c = E(x_1, \dots , x_m)$ po Baconovi metodi (alg. 1). Premisli, kako poimenovati nove spremenljivke in ali se da pogoja iz druge in tretje veje if-stavka malo razrahljati.

Metodo preizkusi na Newtonovem zakonu $F = ma$ (**generiraj_newton**) in Stefanovem zakonu $j = \sigma T^4$ (**generiraj_stefan**).

In [ ]:
def bacon(df, max_iter=20):
    stolpci = list(df.columns)
    for i in range(max_iter):
        df_array = np.array(df)
        """ocenimo, ali obstaja konstanta"""
        sig = np.std(df_array, axis=0)
        # DOPOLNI
        """izracunajmo vrstne rede, najdemo korelacije med njimi"""
        vrstni_redi = df.rank(axis=0)
        korelacije = np.corrcoef(df_array, df_array)
        # DOPOLNI
        """oglej si np.max(korelacije) in np.min(korelacije)"""
        # DOPOLNI
        """pametno poimenuj novi stolpec, da bomo lahko iz njega razbrali enacbo"""
        # DOPOLNI





In [ ]:
podatki = generiraj_newton(100, sum=0)
bacon(podatki)

DODATNO: Če te odkrivanje enačb zanima, si lahko pogledaš še modernejše algoritme. 

**pySR** je enostavna za instalacijo, temelji pa na genetskih algoritmih: https://github.com/MilesCranmer/PySR

**DSO** je trenutno ena od najmočnejših metod za odkrivanje enačb, združuje pa globoko učenje ter genetske algoritme: https://github.com/brendenpetersen/deep-symbolic-optimization